In [ ]:
import numpy
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.domain.create_cartesian(500.*numpy.arange(100), 500.*numpy.arange(30), lat=0, H=50)
sim = pygetm.Simulation(domain, runtype=pygetm.BAROTROPIC_2D, advection_scheme=pygetm.AdvectionScheme.HSIMT)

# Idealized surface forcing
tausx = sim.U.array(fill=0.01)
tausy = sim.V.array(fill=0.)
sp = sim.T.array(fill=0.)
fwf = sim.T.array(fill=0.)

# Time
timestep = 10.
ntime = int(3600. // timestep)

# Set up velocity plot
nsample = 2
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(sim.X.x, sim.X.y, sim.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
u, v = sim.momentum.U.interp(sim.T) / sim.T.D, sim.momentum.V.interp(sim.T) / sim.T.D
Q = ax.quiver(sim.T.x[::nsample, ::nsample], sim.T.y[::nsample, ::nsample], u[::nsample, ::nsample], v[::nsample, ::nsample], scale=.01)
title = ax.set_title('time: 0 s')

In [ ]:
plotting_interval = 5
times = timestep * numpy.arange(ntime)
Us = numpy.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = sim.momentum.U[...].mean()
    sim.update_surface_pressure_gradient(sim.T.z, sp)
    sim.momentum.advance_depth_integrated(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.advance_surface_elevation(timestep, sim.momentum.U, sim.momentum.V, fwf)
    sim.update_depth()

    # Update velocity plot
    if istep % plotting_interval == 0:
        u, v = sim.momentum.U.interp(sim.T) / sim.T.D, sim.momentum.V.interp(sim.T) / sim.T.D
        Q.set_UVC(u[::nsample, ::nsample], v[::nsample, ::nsample])
        title.set_text('time: %s s' % time)
        pc.set_array(sim.T.z[...].ravel())
        fig.canvas.draw()

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(times, Us)
ax.set_title('mean U')
ax.grid()